# 팔머펭귄
> X를 주면 펭귄을 분류하는 문제

     Iter1.
- SVC같은 선형 분류 모델 하나 선정
- 그리드 서치 연습
- 파이프라인 구경

In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv("data_pipeline/penguins_size.csv")
df.sample(15)

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
201,Chinstrap,Dream,49.8,17.3,198.0,3675.0,FEMALE
322,Gentoo,Biscoe,47.2,15.5,215.0,4975.0,FEMALE
223,Gentoo,Biscoe,50.0,15.2,218.0,5700.0,MALE
122,Adelie,Torgersen,40.2,17.0,176.0,3450.0,FEMALE
321,Gentoo,Biscoe,55.9,17.0,228.0,5600.0,MALE
328,Gentoo,Biscoe,43.3,14.0,208.0,4575.0,FEMALE
16,Adelie,Torgersen,38.7,19.0,195.0,3450.0,FEMALE
106,Adelie,Biscoe,38.6,17.2,199.0,3750.0,FEMALE
103,Adelie,Biscoe,37.8,20.0,190.0,4250.0,MALE
188,Chinstrap,Dream,47.6,18.3,195.0,3850.0,FEMALE


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   culmen_length_mm   342 non-null    float64
 3   culmen_depth_mm    342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                334 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [4]:
df['species'].unique()

array(['Adelie', 'Chinstrap', 'Gentoo'], dtype=object)

In [5]:
df['island'].unique()

array(['Torgersen', 'Biscoe', 'Dream'], dtype=object)

In [6]:
df['sex'].unique()

array(['MALE', 'FEMALE', nan, '.'], dtype=object)

In [7]:
penguins=df.dropna()
penguins.reset_index(drop=True, inplace=True)

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,MALE
...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,FEMALE
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


In [10]:
penguins[penguins['sex']=='.']

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
327,Gentoo,Biscoe,44.5,15.7,217.0,4875.0,.


In [11]:
penguins.drop([327], inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_11196\1472391410.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  penguins.drop([327], inplace=True)


In [12]:
penguins['sex'].unique()

array(['MALE', 'FEMALE'], dtype=object)

## SVC로 Pipeline

In [13]:
penguins['species'] = penguins['species'].astype('category')
penguins['species_cod'] = penguins['species'].cat.codes

C:\Users\user\AppData\Local\Temp\ipykernel_11196\3177699378.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  penguins['species'] = penguins['species'].astype('category')
C:\Users\user\AppData\Local\Temp\ipykernel_11196\3177699378.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  penguins['species_cod'] = penguins['species'].cat.codes


In [14]:
penguins_feats=['species','island','culmen_length_mm','culmen_depth_mm',
                'flipper_length_mm','body_mass_g','sex']
penguins_labs=['species_cod']
X,y = penguins[penguins_feats].values, penguins[penguins_labs].values

## SVC를 사용한 학습 및 평가

In [22]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

In [16]:
n_feats = [1,2,3,4]
n_tras = Pipeline(steps=[('scaler', StandardScaler())]) #scaler는 StandardScaler()의 이름으로 선언
c_feats = [1,5]
c_tras = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [17]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num',n_tras,n_feats),
        ('cat',c_tras,c_feats)
    ]
)

In [18]:
pipeline = Pipeline(steps=
                    [('preprocessor', preprocessor),('svc', SVC(probability=True))]
                    )

In [19]:
SVC_scores = []
SVC_fimp = []
SVC_predictions = 0
FOLDS = 5

In [20]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [24]:
for fold, (train_idx, valid_idx) in enumerate(skf.split(penguins[penguins_feats].values, 
                                                        penguins[penguins_labs].values)):
    print(f"Fold = {fold}")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val_train, X_val, y_val_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
multi_model = pipeline.fit(X_val_train, y_val_train)
preds_valid = multi_model.predict(X_val)

acc = accuracy_score(y_val, preds_valid)
SVC_scores.append(acc)
test_preds = multi_model.predict(X_test)

Fold = 0
Fold = 1
Fold = 2
Fold = 3
Fold = 4


ValueError: could not convert string to float: 'Dream'

In [ ]:
np.mean(SVC_scores)
SVC_predictions